In [1]:
import ROOT
from podio import root_io
from math import sqrt

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x6813340
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.ildStyle.SetOptStat(0)
ROOT.ildStyle.SetOptFit(111)

In [4]:
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass_double.h>")
# ROOT.gInterpreter.Declare("#include <marlinutil/GeometryUtil.h>")

True

In [5]:
# Meh would require dd4hep
# Bz = ROOT.MarlinUtil.getBzAtOrigin()
Bz = 3.5
print(Bz)

3.5


In [6]:
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.10.REC_edm4hep.root")
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.1000.REC_edm4hep.root")
reader = root_io.Reader("rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500106.P4f_sw_sl.eL.pR.n000.d_dstm_15176_0.edm4hep.root")

In [7]:
events = reader.get("events")


In [8]:
h_omega = ROOT.TH1D("h_omega", ";pull #Omega; events", 50, -10, 10)
h_phi = ROOT.TH1D("h_phi", ";pull #phi; events", 50, -10, 10)
h_momentum = ROOT.TH1D("h_momentum", "; E [GeV]", 250, 0, 125)
h_omega_2d = ROOT.TH2D("h_omega_2d", "; E [GeV]; pull #Omega", 10, 0, 125, 50, -10, 10)

In [9]:
for event in events:
    # mctruthrecolink = event.get("MCTruthSiTracksLink")
    mctruthrecolink = event.get("MarlinTrkTracksMCTruthLink")
    for link in mctruthrecolink:
        mcp = link.getSim()
        reco = link.getRec()
        weight = link.getWeight()
        # require at least a bit of quality
        # FIXME: ah lol does this direction in the relation kill electrons with high brems loss as the fraction of their initial energy going to the track is lower?
        if (weight < 0.9):
            continue
        # only care about electrons
        if (abs(mcp.getPDG()) != 11 or mcp.getGeneratorStatus() != 1):
            continue
        # require at least some energy
        # FIXME: probably should cut on pT instead
        if (mcp.getEnergy() < 1):
            continue

        reco_track = reco.getTrackStates(0)
        cov = reco_track.covMatrix
        mc_track = ROOT.HelixClass_double()
        mc_pos = mcp.getVertex()
        mc_pos_v = ROOT.std.vector["double"]((mc_pos.x, mc_pos.y, mc_pos.z))
        mc_mom = mcp.getMomentum()
        mc_mom_v = ROOT.std.vector["double"]((mc_mom.x, mc_mom.y, mc_mom.z))
        mc_track.Initialize_VP(mc_pos_v.data(), mc_mom_v.data(), mcp.getCharge(), Bz)
        if (abs(mcp.getCharge()) != 1.0):
            print(f"weird charge: {mcp.getCharge()}")
        pull_omega = (reco_track.omega - mc_track.getOmega()) / sqrt(cov[5])
        pull_phi = (reco_track.phi - mc_track.getPhi0()) / sqrt(cov[2])
        # print(f"energy: {mcp.getEnergy()}")
        # print(f"reco omega: {reco_track.omega}")
        # print(f"mc omega {mc_track.getOmega()}")
        # print(f"sig_omega: {sqrt(cov[5])}")
        # print(f"pull omega: {pull_omega}")
        h_omega.Fill(pull_omega)
        h_phi.Fill(pull_phi)
        mom_v = ROOT.TVector3(mc_mom.x, mc_mom.y, mc_mom.z)
        # h_momentum.Fill(mcp.getEnergy())
        h_momentum.Fill(mom_v.Perp())
        h_omega_2d.Fill(mcp.getEnergy(), pull_omega)



In [10]:
c_omega = ROOT.TCanvas()
h_omega.Draw()
h_omega.Fit("gaus")
c_omega.Draw()
c_omega.SaveAs("plots/pulls_ild/omega.pdf")

c_phi = ROOT.TCanvas()
h_phi.Draw()
h_phi.Fit("gaus")
c_phi.Draw()
c_phi.SaveAs("plots/pulls_ild/phi.pdf")

c_e = ROOT.TCanvas()
h_momentum.Draw()
c_e.Draw()

 FCN=1893.66 FROM MIGRAD    STATUS=CONVERGED      99 CALLS         100 TOTAL
                     EDM=3.85467e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.37679e+03   1.85449e+01   2.61576e-01   8.85471e-06
   2  Mean        -6.78071e-03   1.29695e-02   2.75581e-04   1.14274e-03
   3  Sigma        1.45059e+00   1.46095e-02   3.39190e-05  -5.50606e-02
 FCN=965.927 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=2.4792e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.3 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.54343e+02   1.56463e+01  -1.62073e-01  -6.40081e-07
   2  Mean        -2.56157e-03   1.50722e-02  -8.87341e-06  -1.63764e-03
   3  Sigma        1.

Info in <TCanvas::Print>: pdf file plots/pulls_ild/omega.pdf has been created
Info in <TCanvas::Print>: pdf file plots/pulls_ild/phi.pdf has been created


In [11]:
c_2d = ROOT.TCanvas()
h_omega_2d.Draw("colz")
c_2d.Draw()